# Week 10: 2016/03/28-04/01

## Monday

Happy Easter! No reading today since it's a holiday.

## Tuesday class

Now that we've proven that one language ($A_{\mathsf{TM}}$) is undecidable, we can use it to prove that other languages are undecidable.

### The halting problem

The first example is the halting problem. In many textbooks (and the poem "Scooping the Loop Snooper"), the halting problem is actually the prototypical undecidable language, but Sipser does things a little differently.

The proof is not difficult but it has a weird feel to it. The first thing that you have to remember is that the direction of the reduction is the opposite of what most people intuitively think of first. If you want to show that the halting problem is undecidable, you do _not_ reduce the halting problem to $A_{\mathsf{TM}}$. You assume that the halting problem _is_ decidable, then show via a reduction from $A_{\mathsf{TM}}$ to the halting problem that $A_{\mathsf{TM}}$ would also be decidable, which is a contradiction.

Recall that we designed $U$, a universal TM, that _recognizes_ $A_{\mathsf{TM}}$, but we showed that we cannot design a TM $S$ that _decides_ $A_{\mathsf{TM}}$. Practically, the sticking point is that on input $\langle M, w\rangle$, if $M$ loops on $w$, we have no way to detect it so that we can reject. Enter $R$, a hypothetical TM that decides the halting problem $\mathit{HALT}_{\mathsf{TM}}$. If we had such a machine, then designing $S$ would be easy: Use $R$ to check for looping. if a loop is detected, reject; if no loop is detected, we can safely simulate $M$ and return what it returns. But last time we showed (by diagonalization) that $S$ does not exist. Therefore, $R$ cannot exist either.

### Other properties of Turing machines

Let's take another example not in the chapter; it's Problem 5.12. We say that a Turing machine _erases_ a symbol just when it writes a blank symbol over a nonblank symbol. Show that it is undecidable whether a given Turing machine $M$, when run on input $w$, ever erases a symbol.

First note that it's easy to recognize such situations: just simulate $M$ on $w$ (as in the universal Turing machine) and watch to see whether it erases a symbol. If it does, halt the simulation and accept. If $M$ halts and rejects, reject. If $M$ loops, then loop. But _deciding_ this problem is a whole different story.

Suppose, for the sake of contradiction, that this problem is decidable, that is, there exists a TM $R$ that decides, on input $\langle M, w\rangle$, whether $M$, when run on $w$, ever erases a symbol.

We want to somehow use $R$ to construct a new TM $S$ that decides $A_{\mathsf{TM}}$. If we can do that, we will obtain the desired contradiction (since we known $A_{\mathsf{TM}}$ is decidable).

Slightly flawed, but simple attempt: Given any $M$, we can construct a TM $M'$ that, on input $w$:

1. Simulate $M$ on $w$.
2. If $M$ accepts $w$, then write a symbol, erase it, and accept.
3. Otherwise, do nothing and accept.

Note that if $M$ loops on $w$, then $M'$ also loops on $w$. This TM $M'$ is like an adapter between the two problems at hand. It links the question we want to decide (does $M$ accept $w$?) with the behavior that we have the magic ability to detect (does $M'$ ever erase?).

Then, define $S$ as follows. On input $\langle M, w\rangle$:

1. Use $M$ to construct $M'$ as described above.
2. Run $R$ on $\langle M', w \rangle$.
3. If $R$ accepts, then accept.
4. Otherwise, reject.

If $M$ accepts, then $M'$ writes a symbol and erases it. So $R$, the erasure-detector, accepts, and $S$ should accept too.

If $M$ halts and rejects, then $M'$ does not erase anything. So $R$ rejects, and $S$ should reject too.

If $M$ loops, then $M'$ also loops, and it also doesn't erase anything. So $R$ rejects, and $S$ should reject too.

So, $S$ decides $A_{\mathsf{TM}}$. But we know that $A_{\mathsf{TM}}$ is undecidable, which is a contradiction. We conclude that the erasure-detection problem is also undecidable.

The flaw in the above proof is simply that $M$ itself might erase. And if $M'$ uses its own tape to simulate $M$'s tape, as would be natural to do, then there's a potential bug: if $M$ erases a symbol but ultimately rejects, then $M'$, too, will erase a symbol inadvertently. To fix the flaw, simply define a "virtual blank symbol" $b$, and if $M$ ever erases, then $M'$ should write $b$ instead of a real blank.

### Properties of Turing-recognizable languages

Theorems 5.2 and 3 are slightly different. They are both proofs of the undecidability of questions of the form "Does $L(M)$ belong to such-and-such a language class?". Theorem 5.2 is about the class $\{\emptyset\}$. Theorem 5.3 is about the class of regular languages.

The proofs go like this. Assume that the question is decidable, so there is a TM $R$ that decides it. As before, we need to construct a new TM $S$ that decides $A_{\mathsf{TM}}$. As before, the hard part about designing $S$ is detecting looping, and again the trick is to delegate this part to $R$. Now the proofs become different. Before, $R$ was by assumption a loop detector, so it was easy. Now, $R$ does something seemingly unrelated to loop detection.

So $S$ constructs another TM (called $M_1$ or $M_2$ in the book) to feed into $R$. This machine is like an adapter that gets $R$ to do the dirty work of detecting looping. Crucially, $R$ is able to do its job even if its argument sometimes loops.

In the case of Theorem 5.2 (emptiness), we create $M_1$ to have the following behavior:

- If $M$ accepts $w$, then $M_1$ should recognize $\{w\}$.
- If $M$ rejects or loops on $w$, then $M_1$ should recognize $\emptyset$.

The way that $M_1$ is implemented is, on input $x$:

1. If $x \neq w$, _reject_.
2. Otherwise, run $M$ on input $w$ and _accept_ if $M$ does.

Note that running $M$ is the last step, because of the possibility that $M$ loops. If $M_1$ passes control to $M$ and $M$ loops, then necessarily $M_1$ must loop as well. But $R$ can still do its thing even if $M_1$ loops, and so it has been adapted into a loop detector.

Finally, we can define $S$ to be the machine that runs $R$ on $M_1$ and then returns the opposite answer.

If you like, you could think of $S$ as a Python function...

In [2]:
def is_empty(function): # This is R
    """Returns True iff function recognizes \emptyset."""
    # Insert magic code here

def accepts(function, input): # This is S
    def M1(x):
        """Recognizes {w} if function(input) is True,
           recognizes \emptyset otherwise."""
        if x != input:
            return False
        else:
            return function(input)
    return not is_empty(M1)

In the case of Theorem 5.3 (regularity), we create $M_2$ to have the following behavior:

- If $M$ accepts $w$, then $M_2$ should recognize $\Sigma^\ast$, a regular language.
- If $M$ rejects or loops on $w$, then $M_2$ should recognize $\{\mathtt{0}^n \mathtt{1}^n \mid n \geq 0\}$, a non-regular language.

The way that $M_2$ is implemented is, on input $x$:

1. If $x$ has the form $\mathtt{0}^n\mathtt{1}$, _accept_.
2. Otherwise, run $M$ on input $w$ and _accept_ if $M$ does.

Again note that running $M$ is the last step.

As a Python function...

In [ ]:
def is_regular(function): # This is R
    """Returns True iff function recognizes a regular language."""
    # Insert magic code here

def accepts(function, input): # This is S
    def M2(x):
        """Recognizes \Sigma^\ast if function(input) is True,
           recognizes {0^n 1^n} otherwise."""
        n = len(x)/2
        if x == "0"*n+"1"*n:
            return True
        else:
            return function(input)
    return is_regular(M2)

It turns out that there is a general recipe for proving results of this type. This is called Rice's Theorem and is the focus of one of the homework questions.

### Other undecidable problems

Alas, we don't have time to look at more examples of undecidable problems. The examples we've looked at so far may seem a little arcane because they are all questions about Turing machines, but some other examples are:

- Do two context-free grammars generate the same language (Exercise 5.1)?
- The third and hardest puzzle we did on the first day of class (Section 5.2).
- Does a polynomial equation in several variables have an integral solution?

See [the survey by Poonen](http://www-math.mit.edu/~poonen/papers/sampler.pdf) for more examples.

## Wednesday reading

Catch up by reading Section 5.1, up to but not including "Reductions via Computation Histories." 

If you were interested in the third, hardest puzzle that we did on the first day of class, then read the rest of Section 5.1 and Section 5.2, but this is optional.

## Thursday class

Today we'll discuss a question raised on the first day of class: am I a computer? Most people said no, but now we are armed with a formal definition of what a computer is. So the question becomes: Can my mind/brain be simulated by a Turing machine? This is a thorny question to say the least, but one that's worth at least discussing.

There aren't any notes for this topic save for one "no" argument that is technical. Turing called it "the mathematical objection" and it is usually known today as the Penrose-Lucas argument. This version is due to Penrose and comes from [an article criticizing him](http://www.ihmc.us/users/phayes/pub/lafortehayesford.pdf). 

Let $H$ be a Turing machine that detects _some_ cases of looping. That is, on input $\langle M, w\rangle$, if $H$ accepts, then $M$ loops on input $w$. But if $H$ rejects, it doesn't mean anything.

Now we go through the usual diagonalization argument; the only difference is that it doesn't lead to a contradiction, but to a machine/input pair that is beyond $H$'s detection abilities. Define $D$ to be the Turing machine that, on input $\langle M\rangle$, simulates $H$ on $\langle M, \langle M\rangle\rangle$. If $H$ accepts, then halt. If $H$ rejects, then go into an infinite loop. So,

\begin{equation*}
D(\langle D\rangle) \begin{cases}
\text{halts} & \text{if $H$ thinks $D$ loops on $\langle D\rangle$} \\
\text{loops} & \text{otherwise.}
\end{cases}
\end{equation*}

Since the first case is a contradiction, it must be that $H$ rejects $\langle D, \langle D\rangle\rangle$. So $D$ and $\langle D\rangle$ is an example of a machine/input pair that loops but $H$ is not able to detect.

Suppose that you are equivalent to a Turing machine. It should then be possible to construct a Turing machine $Y$ that accepts $\langle M, w\rangle$ in exactly those cases when you are able to show that $M$ loops on input $w$.

Then, by the above argument, there is a machine/input pair $\langle D, \langle D\rangle\rangle$ that loops, but $Y$ rejects it.  By the definition of $Y$, you are not able to detect that $\langle D, \langle D\rangle\rangle$. But if you understood the above argument, then you know that $\langle D, \langle D\rangle\rangle$ does loop. This is a contradiction!

There are many possible responses to this argument -- what do you think?